In [2]:
import numpy as np 
import pandas as pd 
import sqlite3 as sql 
import json
from tqdm import tqdm 
import datetime as dt


fname_daily = '../data/prices/stocks.db'
fname_intra = '../data/prices/stocks_intraday.db'
stocks_json = '../data/stocks/tickers.json'
 
stocks = json.load(open(stocks_json, 'r'))['all_stocks']

intra_conn = sql.connect(fname_intra)
price_conn = sql.connect(fname_daily)

from Stocks.ta_indicators import Indicators
from Stocks.util import indicator_time_splits 

# lodf = indicator_time_splits(stocks, fname_intra)
lodf = indicator_time_splits(stocks, fname_daily)

print(dt.datetime.today())

Applying Indicators:  95%|█████████▍| 164/173 [00:09<00:00, 22.29it/s]

Error in arm 1m, not enough data.


Applying Indicators: 100%|██████████| 173/173 [00:09<00:00, 17.41it/s]

Error in cava 1m, not enough data.
Error in rddt 3d, not enough data.
Error in rddt 1w, not enough data.
Error in rddt 1m, not enough data.
2024-04-25 20:03:06.355434


In [3]:
### Overbought stocks: Use slow_bb & rsi 
### Oversold stocks: Use slow_bb & rsi

all_signals = {}

for stock in tqdm(stocks): 
    # Copy last row 
    d = lodf[stock].copy()
    for k in d.keys(): 
        od = d[k].copy()
        od.insert(0, 'ticker', stock)
        # Add in Moving Averages (10, 20, 50, 200) period ma
        od['ma10-20'] = od['close'].rolling(window=10).mean() - od['close'].rolling(window=20).mean()
        od['ma20-50'] = od['close'].rolling(window=20).mean() - od['close'].rolling(window=50).mean()
        od['ma50-200'] = od['close'].rolling(window=50).mean() - od['close'].rolling(window=200).mean()
        
        # Record the last row of all signals 
        all_signals[k] = all_signals.get(k, pd.DataFrame()).append(od.tail(1))
       
oversold = {}; overbought = {}

for k in all_signals.keys(): 
    df = all_signals[k].copy()
    # ---------------- BEARISH CONDITIONS ---------------- 
    # If RSI > 70 and BB percent > 75: Overbought 
    ob = df[ (df['bb_percent'] > .75) & (df['ma10-20'] < 0)]
    overbought[k] = list(ob.ticker)
    # ---------------- BULLISH CONDITIONS ---------------- 
    # If RSI < 30 and BB percent < 25: Oversold
    os = df[(df['bb_percent'] < .25) | (df['ma10-20'] > 0)]
    oversold[k] = list(os.ticker)
    
print('Overbought: (Bearish)')
print(overbought)
print('Oversold: (Bullish)')
print(oversold)
    

100%|██████████| 173/173 [00:02<00:00, 69.04it/s] 

Overbought: (Bearish)
{'3d': ['mo', 'baba', 'cmre', 'ko', 'bil'], '1w': ['ccj', 'hsbc', 'weat'], '1m': ['rtx', 'dis', 'oxy']}
Oversold: (Bullish)
{'3d': ['gdx', 'amzn', 'spy', 'xlf', 'pfe', 'igv', 'uber', 'xli', 'uup', 'pep', 'goog', 'mmm', 'ntr', 'dkng', 'xlu', 'pltr', 'sq', 'vxx', 'cvx', 'fcel', 'intc', 'smh', 'mu', 'hd', 'shy', 'rrc', 'enph', 'fdn', 'lc', 'bkkt', 'bno', 'lmt', 'wfc', 'crwd', 'mrk', 'slv', 'qcom', 'ardx', 'mos', 'qs', 'msft', 'tgt', 'meta', 'abbv', 'moo', 'rblx', 'qqq', 'iei', 'nflx', 'pbw', 'ccj', 'xme', 'ebay', 'f', 'ief', 'sono', 'pypl', 'ibm', 'xly', 'amd', 'xle', 'xom', 'xlb', 'xlk', 'xlre', 'xlv', 'ibb', 'lqd', 'jets', 'tlt', 'xlc', 'hsbc', 'kr', 'rtx', 'afrm', 'gsl', 'wba', 'ung', 'ba', 'ge', 'xrt', 'ctva', 'corn', 'aa', 'dis', 'csco', 'cvs', 'chgg', 'icln', 'ctra', 'fang', 'li', 'run', 'tsn', 'iwm', 'spwr', 'itb', 'oxy', 'tbt', 'c', 'gld', 'lcid', 'weat', 'x', 'tan', 'eog', 'jpm', 'k', 'tmv', 'sony', 'hood', 'spot', 'cag', 'uvxy', 'ual', 'pgr', 'afl', 'aig', 

In [15]:
import json 

all_signals['3d'].to_csv('../webpage/dev/data/stock_signals.csv', index=False)

In [16]:
# Find the common stocks within the overbought dictionary 
# bearish = set(overbought['5min']).intersection(set(overbought['15min'])).intersection(set(overbought['30min'])).intersection(set(overbought['1h']))
# bullish = set(oversold['5min']).intersection(set(oversold['15min'])).intersection(set(oversold['30min'])).intersection(set(oversold['1h']))

## Count the number of times a stock is overbought
from collections import Counter
c = Counter()
for k in overbought.keys(): 
    c.update(overbought[k])
    
common_bearish = [x for x in c.keys() if c[x] >= 1]

c = Counter()
for k in oversold.keys(): 
    c.update(oversold[k])

common_bullish = [x for x in c.keys() if c[x] >= 2]


bearish = common_bearish.copy()
bullish = common_bullish.copy()

# Potential Tweets: 
### Bullish 
t0 = 'Bullish Stocks:\n' + '\t'.join(['$' + x.upper() for x in bullish])
t1 = 'Bearish Stocks:\n' + '\t'.join(['$' + x.upper() for x in bearish])

print(t0)
print('\n')
print(t1)


Bullish Stocks:
$GDX	$AMZN	$SPY	$XLF	$PFE	$IGV	$UBER	$XLI	$UUP	$PEP	$GOOG	$MMM	$NTR	$DKNG	$XLU	$PLTR	$SQ	$CVX	$FCEL	$INTC	$SMH	$MU	$HD	$SHY	$RRC	$FDN	$LC	$BKKT	$BNO	$LMT	$WFC	$CRWD	$MRK	$SLV	$QCOM	$ARDX	$MOS	$QS	$MSFT	$TGT	$META	$ABBV	$MOO	$RBLX	$QQQ	$IEI	$NFLX	$PBW	$CCJ	$XME	$EBAY	$F	$IEF	$SONO	$PYPL	$IBM	$XLY	$AMD	$XLE	$XOM	$XLB	$XLK	$XLRE	$XLV	$IBB	$LQD	$JETS	$TLT	$XLC	$HSBC	$KR	$RTX	$AFRM	$GSL	$WBA	$BA	$GE	$XRT	$CTVA	$CORN	$AA	$DIS	$CSCO	$CVS	$CHGG	$ICLN	$CTRA	$FANG	$LI	$RUN	$TSN	$IWM	$SPWR	$ITB	$OXY	$TBT	$C	$GLD	$LCID	$X	$TAN	$EOG	$JPM	$K	$TMV	$SONY	$HOOD	$SPOT	$CAG	$UAL	$PGR	$AFL	$AIG	$MPW	$DIA	$ARM	$TTD	$AMAT	$DASH	$OPEN	$TDOC	$SMCI	$HIMS	$CAVA	$GME	$AAPL	$UWMC	$WMT	$VALE	$JKS	$AMC	$BITO	$CLSK	$COIN	$ASAN	$SBUX	$CMRE	$INTU	$NVDA	$SNOW	$LULU	$ABNB	$V	$PHO	$BIL	$AVGO	$DJT


Bearish Stocks:
$MO	$BABA	$CMRE	$KO	$BIL	$CCJ	$HSBC	$WEAT	$RTX	$DIS	$OXY


### Potential Problems: 
1. Stair-Stepping
   1. This happens when there is a lot of demand for a stock, and price is increasing rapidly after short choppy periods of consolidation.
      1. This actually describes the Action of the SPY in the past few weeks (as of 3/25/2024). 

In [17]:
overbought

{'3d': ['mo', 'baba', 'cmre', 'ko', 'bil'],
 '1w': ['ccj', 'hsbc', 'weat'],
 '1m': ['rtx', 'dis', 'oxy']}

In [18]:
oversold

{'3d': ['gdx',
  'amzn',
  'spy',
  'xlf',
  'pfe',
  'igv',
  'uber',
  'xli',
  'uup',
  'pep',
  'goog',
  'mmm',
  'ntr',
  'dkng',
  'xlu',
  'pltr',
  'sq',
  'vxx',
  'cvx',
  'fcel',
  'intc',
  'smh',
  'mu',
  'hd',
  'shy',
  'rrc',
  'enph',
  'fdn',
  'lc',
  'bkkt',
  'bno',
  'lmt',
  'wfc',
  'crwd',
  'mrk',
  'slv',
  'qcom',
  'ardx',
  'mos',
  'qs',
  'msft',
  'tgt',
  'meta',
  'abbv',
  'moo',
  'rblx',
  'qqq',
  'iei',
  'nflx',
  'pbw',
  'ccj',
  'xme',
  'ebay',
  'f',
  'ief',
  'sono',
  'pypl',
  'ibm',
  'xly',
  'amd',
  'xle',
  'xom',
  'xlb',
  'xlk',
  'xlre',
  'xlv',
  'ibb',
  'lqd',
  'jets',
  'tlt',
  'xlc',
  'hsbc',
  'kr',
  'rtx',
  'afrm',
  'gsl',
  'wba',
  'ung',
  'ba',
  'ge',
  'xrt',
  'ctva',
  'corn',
  'aa',
  'dis',
  'csco',
  'cvs',
  'chgg',
  'icln',
  'ctra',
  'fang',
  'li',
  'run',
  'tsn',
  'iwm',
  'spwr',
  'itb',
  'oxy',
  'tbt',
  'c',
  'gld',
  'lcid',
  'weat',
  'x',
  'tan',
  'eog',
  'jpm',
  'k',
  'tmv